# Intro and data creation

In [1]:
import dask.array as da
import dask.dataframe as dd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.tsa.api as smt

from arch import arch_model

In [2]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

<IPython.core.display.Javascript object>

In [3]:
OUTPUT_PATH = "./simulated_data/"

In [4]:
def GARCH_sim(theta, n=1000, n_iter = 1000):
    (mu,omega,alpha,beta) = theta
    nu=np.random.normal(size=(n + n_iter,))
    sigma=np.zeros((n + n_iter,))
    eps=np.zeros((n + n_iter,))
    
    for i in range(1,n+n_iter):
        sigma[i]=np.sqrt(omega+alpha*eps[i-1]**2+beta*sigma[i-1]**2)
        eps[i]=sigma[i]*nu[i]
        
    return eps[n_iter:], sigma[n_iter:]

In [5]:
returns_df=pd.read_excel('DATA_MARKETS.xlsx', 'DAILY',usecols="B:AI")
for col in returns_df.columns:
    returns_df[col]=1000*returns_df[col]
returns=np.array(returns_df)
print("Done")

Done


In [6]:
returns_df.head()

,EQ_UK,EQ_SZ,EQ_CA,EQ_JN,EQ_AS,EQ_EU,EQ_SCEU,EQ_US,EQ_SCUS,EQ_GR,...,GOV_SZ,GOV_JN,GOV_CA,GOV_US,CORP_EU,CORP_US,HY_EU,HY_US,CONV_US,D_EM
0,-9.801320,-4.169441,-3.791472,2.977168,13.074797,-5.571522,-0.225441,0.096613,-8.857993,-4.908900,...,-4.686769,1.901668,-7.509960,-6.798712,-4.044220,-5.180136,-1.493298,-4.414943,0.304094,2.985870
1,13.767230,18.797430,16.008449,-0.815770,-5.119100,13.233616,5.750042,17.019366,7.266947,18.403567,...,-1.550899,5.591229,2.770936,3.546833,-0.937864,2.610108,0.205971,1.447437,9.777936,5.347771
2,5.220638,-1.269995,-7.359811,-3.708973,8.721848,8.557060,4.203800,-2.452911,-1.429784,15.901940,...,-0.161656,-1.535597,-5.756831,-6.518510,1.206957,-3.907483,1.754873,-1.565325,3.793803,-0.702085
3,2.136577,3.643378,5.012538,4.827877,-8.656507,4.589546,0.613976,18.646517,9.018593,9.707932,...,-6.966412,-6.856733,-1.223358,-0.491404,-1.246716,-0.137288,0.250257,0.220775,10.555790,6.025640
4,16.875769,5.794007,5.401743,7.737938,5.516345,7.556675,5.299269,-5.834126,-3.866476,12.326930,...,-4.148291,-2.806796,-1.492422,0.145416,0.082530,-0.302883,0.419969,1.218183,-1.574365,4.243624


In [7]:
garch_param_means=[]
garch_param_chol=[]
for col in returns_df.columns:
    garch = arch_model(returns_df[col]).fit(disp='off')
    garch_param_means.append(np.array(garch.params))
    garch_param_chol.append(np.linalg.cholesky(garch.param_cov))
garch_param_means, garch_param_chol

C:\ProgramData\Anaconda3\lib\site-packages\arch\univariate\base.py:709: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  ConvergenceWarning,


([array([0.41710866, 1.77952052, 0.10735364, 0.88005783]),
  array([0.50538994, 2.78958192, 0.1164304 , 0.8620654 ]),
  array([0.47405369, 0.82987277, 0.07583916, 0.91677374]),
  array([0.53436259, 4.56104754, 0.10233185, 0.87478239]),
  array([0.62806078, 0.94017615, 0.07954585, 0.91177422]),
  array([0.74570702, 2.17156281, 0.09397777, 0.89501166]),
  array([0.92693744, 1.84441496, 0.12738568, 0.86120927]),
  array([0.58635354, 1.67932743, 0.09023901, 0.89596195]),
  array([0.70522347, 3.78506464, 0.07998686, 0.89932329]),
  array([0.83209791, 2.38924298, 0.08787944, 0.90134717]),
  array([0.72953643, 2.43825112, 0.09621223, 0.89382933]),
  array([0.44342745, 1.59503692, 0.09362977, 0.90357443]),
  array([0.81823918, 2.36682482, 0.09455528, 0.89764098]),
  array([0.90834686, 3.20556137, 0.10372649, 0.88384723]),
  array([0.70507942, 5.58998934, 0.05821624, 0.92275292]),
  array([0.99670905, 5.33543169, 0.08080402, 0.90487119]),
  array([0.54889104, 0.69463647, 0.06281003, 0.93535855]

In [ ]:
N_years_length=365
N_years_sim=30
N_sim=500
N_assets=len(returns_df.columns)

for i_year in range(N_sim):
    output_df=pd.DataFrame(columns=returns_df.columns)
    for i_asset,col in enumerate(returns_df.columns):
        sim_params=np.maximum(garch_param_means[i_asset]+np.dot(np.random.normal(size=(4)),garch_param_chol[i_asset].T),np.zeros_like(4))
        val, vol = GARCH_sim(sim_params,N_years_sim*N_years_length,1000)
        output_df[col]=val/1000
    output_df.to_csv(path_or_buf=OUTPUT_PATH+"simulation_"+str(i_year)+".csv",header=True,index=False)

# Dask kicks in

## 1) basic operations

In [8]:
import dask
from dask.distributed import Client
import dask.dataframe as dd

client = Client(n_workers=3, threads_per_worker=2, memory_limit='1GB')
client

Client Scheduler: tcp://127.0.0.1:52853 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 3 Cores: 6 Memory: 3.00 GB


In [9]:
dfs = dd.read_csv(OUTPUT_PATH+'simulation_*.csv')

In [10]:
dfs.npartitions

100

In [11]:
asset_means,asset_corr = dfs.mean(axis=0), dfs.corr()
asset_means,asset_corr

(Dask Series Structure:
 npartitions=1
 CONV_US    float64
 HY_US          ...
 dtype: float64
 Dask Name: dataframe-mean, 303 tasks, Dask DataFrame Structure:
                  EQ_UK    EQ_SZ    EQ_CA    EQ_JN    EQ_AS    EQ_EU  EQ_SCEU    EQ_US  EQ_SCUS    EQ_GR    EQ_FR    EQ_IT    EQ_SP    EM_IN    EM_BR    EM_RU    EM_KO    EM_TW    EM_CN   GOV_UK   GOV_SP   GOV_FR   GOV_GR   GOV_IT   GOV_SZ   GOV_JN   GOV_CA   GOV_US  CORP_EU  CORP_US    HY_EU    HY_US  CONV_US     D_EM
 npartitions=1                                                                                                                                                                                                                                                                                                                  
 EQ_UK          float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  fl

In [12]:
asset_means,asset_corr = dask.compute(asset_means,asset_corr)

In [13]:
asset_means

EQ_UK      9.379249e-09
EQ_SZ     -1.301480e-05
EQ_CA     -4.168823e-06
EQ_JN      1.075309e-05
EQ_AS     -6.427503e-06
EQ_EU     -2.204249e-05
EQ_SCEU    2.239920e-05
EQ_US     -5.391089e-06
EQ_SCUS    7.695105e-06
EQ_GR     -1.453334e-06
EQ_FR     -1.887423e-05
EQ_IT     -3.402993e-04
EQ_SP     -7.839031e-06
EM_IN     -1.396936e-04
EM_BR     -2.840010e-06
EM_RU      3.936591e-03
EM_KO     -3.491617e-02
EM_TW      4.769662e-05
EM_CN     -2.286743e-05
GOV_UK     3.793526e-06
GOV_SP    -9.788156e-06
GOV_FR    -6.462501e-07
GOV_GR     1.072908e-06
GOV_IT     1.804302e+07
GOV_SZ    -2.304960e-06
GOV_JN     2.103658e-02
GOV_CA     1.838521e-06
GOV_US    -4.422951e-06
CORP_EU    2.142186e-06
CORP_US    4.470381e-06
HY_EU      4.717881e+32
HY_US     -9.770125e+42
CONV_US    7.171465e-06
D_EM       2.471132e-03
dtype: float64

In [14]:
asset_corr

,EQ_UK,EQ_SZ,EQ_CA,EQ_JN,EQ_AS,EQ_EU,EQ_SCEU,EQ_US,EQ_SCUS,EQ_GR,...,GOV_SZ,GOV_JN,GOV_CA,GOV_US,CORP_EU,CORP_US,HY_EU,HY_US,CONV_US,D_EM
EQ_UK,1.000000e+00,0.000213,-0.000579,-0.000989,-0.001704,0.001380,0.000415,-0.000606,4.556449e-05,0.000602,...,0.000588,-8.126651e-04,2.111832e-07,-0.000751,-0.002446,0.001634,7.208442e-04,-1.648424e-04,-0.000925,4.664013e-04
EQ_SZ,2.133989e-04,1.000000,0.000840,-0.001035,-0.000177,0.000399,-0.000740,-0.001124,9.675269e-04,0.000760,...,0.000918,-9.693953e-04,1.326860e-03,0.001494,-0.002012,0.000113,-7.955336e-04,-2.799806e-04,0.000247,1.096224e-03
EQ_CA,-5.792878e-04,0.000840,1.000000,0.000446,-0.000935,0.000244,-0.000080,-0.000087,1.417541e-03,0.000934,...,0.000668,1.045323e-03,-9.628834e-04,-0.000113,-0.000290,0.000328,3.975184e-03,5.588769e-04,0.001006,1.155802e-04
EQ_JN,-9.889690e-04,-0.001035,0.000446,1.000000,0.000421,-0.001905,0.000562,-0.000485,1.018473e-03,0.000843,...,-0.000368,1.816353e-03,-1.461237e-03,-0.000554,0.000419,0.000396,2.310873e-03,-1.058223e-03,-0.001085,5.099730e-04
EQ_AS,-1.704230e-03,-0.000177,-0.000935,0.000421,1.000000,0.001592,-0.000044,0.000932,-6.825709e-04,-0.000466,...,0.000075,9.957025e-04,1.018467e-03,-0.002702,-0.000404,0.000570,-3.753055e-04,-7.036532e-04,0.000784,1.284450e-03
EQ_EU,1.380351e-03,0.000399,0.000244,-0.001905,0.001592,1.000000,0.000620,0.000853,9.624932e-04,0.001199,...,0.000006,-5.202573e-04,8.097476e-04,0.000279,0.000434,0.001886,-4.513768e-04,1.328793e-03,0.000498,-2.635975e-05
EQ_SCEU,4.145675e-04,-0.000740,-0.000080,0.000562,-0.000044,0.000620,1.000000,0.000113,2.612797e-04,-0.000424,...,-0.000511,-5.165338e-04,2.364682e-03,-0.001447,-0.000080,-0.001095,-2.741080e-04,3.245466e-04,-0.000103,5.125371e-04
EQ_US,-6.055293e-04,-0.001124,-0.000087,-0.000485,0.000932,0.000853,0.000113,1.000000,1.253086e-04,-0.001896,...,-0.000058,1.416094e-04,3.611059e-04,0.000126,-0.000782,0.001050,-7.754452e-04,7.881796e-04,-0.000279,-9.656991e-06
EQ_SCUS,4.556449e-05,0.000968,0.001418,0.001018,-0.000683,0.000962,0.000261,0.000125,1.000000e+00,-0.001405,...,-0.000796,-6.134116e-04,-1.968765e-03,0.000314,0.000049,0.001921,-1.512784e-03,-6.589733e-04,-0.000483,9.408836e-04
EQ_GR,6.022869e-04,0.000760,0.000934,0.000843,-0.000466,0.001199,-0.000424,-0.001896,-1.404655e-03,1.000000,...,0.000055,-3.927913e-04,-1.044311e-03,-0.000231,-0.000887,-0.000436,6.255666e-04,-3.431078e-04,0.001523,1.948937e-03


## 2) distributed learning with sklearn
### a) dumb linear model

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn import ensemble
import joblib

In [16]:
client.close()

In [17]:
client2 = Client(n_workers=8, threads_per_worker=1, memory_limit='1GB')
client2

Client Scheduler: tcp://127.0.0.1:52909 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 8 Memory: 8.00 GB


In [18]:
target_col = dfs.columns[0]
data_cols = dfs.columns[1:]

In [19]:
ridge_reg = linear_model.Ridge(alpha=.5)
ridge_reg

Ridge(alpha=0.5)

In [20]:
with joblib.parallel_backend('dask'):
    ridge_reg.fit(dfs[data_cols], dfs[target_col])

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_ridge.py:148: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number2.528473e-101
  overwrite_a=True).T


In [21]:
ridge_reg.intercept_, ridge_reg.coef_

(2.877295146045534e-08,
 array([ 2.20332561e-04, -6.32130247e-04, -8.40364029e-04, -1.71265997e-03,
         1.14846544e-03,  2.16304873e-04, -6.41251286e-04,  3.89668034e-05,
         4.46895823e-04, -4.08035389e-04,  1.05396036e-06,  9.30835403e-05,
        -2.93911834e-05,  5.26256894e-04, -1.48577049e-06, -3.57559949e-07,
         3.68724216e-04,  5.79368201e-05, -3.41903392e-04,  2.22234063e-04,
         2.04584562e-03,  2.90343589e-03, -3.51690166e-16,  2.93229914e-03,
        -8.05167673e-07, -1.12456286e-05, -2.11223675e-03, -1.64380472e-02,
         4.65759157e-03,  1.74236186e-41, -6.34626816e-54, -1.79526037e-03,
         2.22252553e-06]))

In [22]:
with joblib.parallel_backend('dask'):
    print("r2 =", ridge_reg.score(dfs[data_cols], dfs[target_col]))

r2 = 2.5726597027708742e-05


### b) bayesian ridge

In [ ]:
bridge_reg = linear_model.BayesianRidge()
bridge_reg

In [ ]:
with joblib.parallel_backend('dask'):
    bridge_reg.fit(dfs[data_cols], dfs[target_col])
bridge_reg.intercept_, bridge_reg.coef_

In [ ]:
with joblib.parallel_backend('dask'):
    print("r2 =", bridge_reg.score(dfs[data_cols], dfs[target_col]))

## 3) distributed learning with dask_ml

In [23]:
import dask_ml.datasets
import dask_ml.cluster
import dask_ml.linear_model
import dask_ml.xgboost

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.cluster.k_means_ module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.cluster. Anything that cannot be imported from sklearn.cluster is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\dask\array\random.py:27: FutureWarning: dask.array.random.doc_wraps is deprecated and will be removed in a future version
  FutureWarning,
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
C:

### a) linear regression 2.0

In [ ]:
lr = dask_ml.linear_model.LinearRegression()
lr.fit(dfs[data_cols].to_dask_array(lengths=True), dfs[target_col].to_dask_array(lengths=True))

In [ ]:
lr.intercept_ , lr.coef_

In [ ]:
lr.score(dfs[data_cols], dfs[target_col])

### b) XGBoost

In [25]:
model = dask_ml.xgboost.XGBRegressor()

In [ ]:
model.fit(dfs[data_cols].to_dask_array(lengths=True), dfs[target_col].to_dask_array(lengths=True))